In [151]:
import pickle
import os
from datetime import datetime
import pandas as pd
import numpy as np

In [125]:
model_path = r"D:\projects\seoul bike sharing demand prediction\Models\XGBoost_regressor_r2_0_892_v1.pkl"
model = pickle.load(open(model_path, 'rb'))

In [129]:
data = [
    [-0.50839691,  1.1998884 ,  0.52836228,  0.06680005,  0.9210637 ,
        0.14200505, -0.13401915, -0.17102636,  0.22996655, -0.18708879,
       -0.87815509,  1.01127504,  0.30179287, -0.58569899,  1.73073366,
       -0.56834235,  2.53140945, -0.41106465, -0.42003701, -0.40559398,
       -0.41153889, -0.41272349],

    [ 0.07092757,  1.65407403, -0.35217637,  0.3562486 , -1.04340087,
        3.01478263, -0.13401915, -0.17102636,  0.22996655, -0.18708879,
        0.48566198,  0.14070562,  0.30179287, -0.58569899,  1.73073366,
       -0.56834235, -0.39503684, -0.41106465, -0.42003701, -0.40559398,
       -0.41153889, -0.41272349]
]

In [130]:
prediction = model.predict(data)
list(prediction)

[1816.5343, 747.21136]

In [131]:
# load sc pickle

In [132]:
scaling_dump_path = "D:\projects\seoul bike sharing demand prediction\Models\sc.pkl"
scaler = pickle.load(open(scaling_dump_path, 'rb'))

In [133]:
scaler

StandardScaler()

In [134]:
# user Input

date = "01/02/2025"
hour = 22
temperature = 20
humidity = 45
wind_speed = 69
visibility = 29
solar_Radiation = 0.0
rainfall = 0.9
snowfall = 0.2
seasons = "Winter"
holiday = "No Holiday"
functioning_Day =  "Yes"

In [135]:
# converting user data into model consumable format

In [136]:
holiday_dic = {"Holiday": 0, "No Holiday": 1}
functioning_day_dic = {"Yes": 0, "No": 1}

def get_string_to_datetime(date):
    dt = datetime.strptime(date, '%d/%m/%Y')
    return {'day': dt.day, 'month' : dt.month, 'year' : dt.year, 'week_day' : dt.strftime("%A")}

str_to_date = get_string_to_datetime(date)
str_to_date

{'day': 1, 'month': 2, 'year': 2025, 'week_day': 'Saturday'}

In [137]:
u_input_list = [hour, temperature, humidity, wind_speed, visibility, solar_Radiation, rainfall, snowfall, holiday_dic[holiday],
                functioning_day_dic[functioning_Day],
                str_to_date['day'], str_to_date['month'], str_to_date['year'],]

features_name = ['Hour', 'Temperature(°C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day',
       'Month', 'Year',]

df_u_input = pd.DataFrame([u_input_list], columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,22,20,45,69,29,0.0,0.9,0.2,1,0,1,2,2025


In [138]:
def seasons_to_df(seasons):
    seasons_cols = ['Spring', 'Summer', 'Winter']
    seasons_data = np.zeros((1, len(seasons_cols)), dtype= "int")
    df_seasons = pd.DataFrame(seasons_data, columns=seasons_cols)
    if seasons in seasons_cols:
        df_seasons[seasons]=1
    return df_seasons
    
df_seasons = seasons_to_df(seasons)
df_seasons

,Spring,Summer,Winter
0,0,0,1


In [143]:
def days_to_df(week_day):
    days_names = ['Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_data = np.zeros((1, len(days_names)), dtype="int")
    df_days = pd.DataFrame(days_data,columns=days_names)
    if week_day in days_names:
        df_days[week_day]=1
    return df_days
    
df_days = days_to_df(str_to_date["week_day"])
df_days

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,1,0,0,0,0


In [144]:
df_for_pred= pd.concat([df_u_input, df_seasons, df_days], axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,22,20,45,69,29,0.0,0.9,0.2,1,0,...,2025,0,0,1,0,1,0,0,0,0


In [146]:
sc_data_for_pred = scaler.transform(df_for_pred)
sc_data_for_pred

array([[ 1.51923877,  0.59430757, -0.64568926, 64.90327379, -2.30662204,
        -0.65916799,  0.70941089,  0.27197964,  0.22996655, -0.18708879,
        -1.67371506, -1.31024342, 25.60905941, -0.58569899, -0.57778965,
         1.75950288, -0.39503684,  2.43270737, -0.42003701, -0.40559398,
        -0.41153889, -0.41272349]])

In [147]:
## prediction in deployment

In [148]:
model.predict(sc_data_for_pred)

array([394.7972], dtype=float32)

In [150]:
print(f"Rented Bike Demand on date : {date} and time : {hour} is : {round(model.predict(sc_data_for_pred).tolist()[0])}")

Rented Bike Demand on date : 01/02/2025 and time : 22 is : 395
